# Demo notebooks to get OpenAI API keys.
Reference: [PathAI OpenAI access](https://pathai.atlassian.net/wiki/spaces/ML/pages/149553179/OpenAI+API+Access)

To run this noteobook, use the `base` environment with Python 3.10.14.

In [1]:
import os
import hvac
from functools import lru_cache
from urllib.parse import quote, urlencode

In [2]:
VAULT_HOSTNAME = "vault-server.services.pathai.com"
VAULT_TOKEN_PATH = "secret/openai-api"

In [4]:
def fetch_vault_token() -> str:
    """Returns a vault token that is assumed to live at '~/.vault-token'."""
    with open(os.path.join(os.path.expanduser("~"), ".vault-token"), encoding="utf-8") as vault_file:
        return vault_file.readline().strip()

In [5]:
@lru_cache()
def fetch_openai_token_from_vault(use_case="VLM"):
    """Fetch a GitLab token from vault given a vault path.
    Args:
        ml_platform: Whether or not to retrieve token for ML Platform (vs. ML Plug-ins). Defaults to True.
    Returns:
        GitLab token from vault.
    """
    response = None
    try:
        vault_token = fetch_vault_token()
        client = hvac.Client(f"https://{quote(VAULT_HOSTNAME)}", token=vault_token, verify=False)
        response = client.read(VAULT_TOKEN_PATH)
    finally:
        if not response:
            raise ValueError(
                "Fetch vault token failed, try refreshing vault token using "
                "'vault login -method=okta username=your.name@pathai.com'"
            )
    if use_case not in response['data']:
        raise ValueError(f"No access-token found for use-case {use_case}")
    return response['data'][use_case]

In [9]:
os.environ['OPENAI_API_KEY'] = fetch_openai_token_from_vault()

# Test the API token

In [14]:
from openai import OpenAI

In [17]:
# print(f"OpenAI API Key: {os.environ['OPENAI_API_KEY']}")

In [15]:
client = OpenAI()
response = client.chat.completions.create(
  model="gpt-4o-2024-05-13",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Who won the world series in 2021?"}
  ]
)
print(response.choices[0].message.content)


{
  "year": 2021,
  "winner": "Atlanta Braves"
}
